In [66]:
from scipy.special import erfinv
from collections import OrderedDict
from math import sqrt
import numpy as np

In [67]:
def cdfinv(y):
    """简化的公式，与原NormalCDFInverse等价，且精度更高

    公式推导参见https://www.cnblogs.com/htj10/p/8621771.html
    """
    return sqrt(2) * erfinv(2 * y - 1)

In [68]:
def rankGaussTrafo(dataIn):
    hist = dict()       # hist统计元素的出现频率
    for i in dataIn:
        if i not in hist:
            hist[i] = 1
        else:
            hist[i] += 1

    hist = OrderedDict([t for t in sorted(hist.items(), key=lambda d:d[0])])    # 按照key排序

    trafoMap = dict()
    if len(hist) == 1:      # unary column: trafo all to 0
        trafoMap[list(hist.keys())[0]] = 0.0
    elif len(hist) == 2:    # binary column: trafo to 0 / 1
        trafoMap[list(hist.keys())[0]] = 0.0
        trafoMap[list(hist.keys())[1]] = 1.0
    else:                   # more than 2 unique values
        mean = 0.0
        cnt = 0
        N = len(dataIn)

        for key, value in hist.items():
            rankV = cnt * 1.0 / N       # 累计次数占总次数的比例，取值[0,1]，单调递增（和分布函数F的性质对应）

            rankV = rankV * 0.998 + 1e-3    # 注意到cdfinv(0) = -inf，而cdfinv(1) = inf。这个操作使得rankV限制在[0.001,0.999]，而cdfinv(rankV)限制在[-3.09,3.09]，避免了极端情况的发生

            scale_factor = 0.7      # 使用0.7可以得到原cpp中的结果，对分布有收缩作用。建议使用1.0，可保持std=1.0
            rankV = cdfinv(rankV) * scale_factor     # 将其作为分布函数F的值，逆向求N(0,1)的α分位数

            mean += value * rankV   # value是出现次数，乘以rankV。rankV可以看作是value的权重。注意到hist是按key从小到大排序的，排位越后的数对均值的贡献越大
            trafoMap[key] = rankV   # 记录为trafoMap的值
            cnt += value            # 累计次数
        
        mean /= N

        for key in trafoMap.keys():
            trafoMap[key] -= mean   # 每个rankV减去均值，得到最终trafoMap

    dataOut = dataIn.copy()
    for i in range(len(dataIn)):    # 这里简单地把trafoMap映射到输出
        dataOut[i] = trafoMap[dataIn[i]]
    return dataOut

In [69]:
data = [-19.9378,10.5341,-32.4515,33.0969,24.3530,-1.1830,-1.4106,-4.9431,
        14.2153,26.3700,-7.6760,60.3346,36.2992,-126.8806,14.2488,-5.0821,
        1.6958,-21.2168,-49.1075,-8.3084,-1.5748,3.7900,-2.1561,4.0756,
        -9.0289,-13.9533,-9.8466,79.5876,-13.3332,-111.9568,-24.2531,120.1174]

dataOut = rankGaussTrafo(data)
print(dataOut)

[-0.5517799966896523, 0.4090010366917574, -0.8516215614806684, 0.7725859073647011, 0.6097608726396847, 0.17749425578711678, 0.12237751864416965, -0.042296592392275054, 0.4720838737474867, 0.686977660084494, -0.15498726783224498, 0.9868192248755101, 0.8702983648910773, -2.095563782620048, 0.5386408376293267, -0.09810428241439527, 0.23330194580923708, -0.637388243969859, -1.0013321669392468, -0.21338178582526862, 0.06759883169742091, 0.29018493122708683, 0.012820144750672259, 0.34857944922011036, -0.27380337329691556, -0.4745632092448429, -0.3368862103526449, 1.1365298303340896, -0.4034431742344851, -1.2271028375799966, -0.7351007014962355, 1.362300500974838]


In [70]:
# 如果scale_factor=0.7，则最后数据std也会变成约0.7，而不是1.0
np.mean(dataOut), np.std(dataOut)

(5.204170427930421e-18, 0.7255313511486773)

# output from cpp version (scale_factor=0.7)
-0.551684
0.408716
-0.851753
0.772574
0.609613
0.177273
0.122247
-0.0420627
0.471833
0.686894
-0.154673
0.986964
0.870363
-2.09576
0.538436
-0.0978157
0.233026
-0.637364
-1.00154
-0.213066
0.0676051
0.289884
0.012963
0.348277
-0.273506
-0.474402
-0.336622
1.13675
-0.403226
-1.22739
-0.735153
1.3626
